# Test functions

1D, without noise

In [ ]:
xx = np.linspace(-2, 10, 250).reshape(-1,1)

def target(xx):
    return np.exp(-(xx - 2)**2) + np.exp(-(xx - 6)**2/10) + 1/ (xx**2 + 1)

params = {'ucb': {'kappa': 5, 'xi': 0}, 'ei': {'kappa': 5, 'xi': 0}, 'poi': {'kappa': 5,'xi': 0.01}, 'kg': {'kappa': 5, 'xi': 0}}
optimizers = {}
for acq in params.keys():
    optimizers[acq] = BayesianOptimization(target, {'x': (min(xx), max(xx))}, random_state=27)

In [ ]:
def update_optimizers(n_iter, optimizers, init_points=2):
    
    if len(list(optimizers.values())[0]._space.target) == 0:
        for acq in optimizers.keys():
            optimizers[acq].maximize(init_points=init_points, n_iter=0, acq=acq)
    else:
        for acq in optimizers.keys():
            optimizers[acq].maximize(init_points=0, n_iter=n_iter, acq=acq)
    return optimizers   

In [ ]:
optimizers = update_optimizers(0, optimizers, 5)
#plot_gp(optimizers, xx, target, params, 5)

In [ ]:
optimizers = update_optimizers(3, optimizers)
plot_gp(optimizers, xx, target, params, 5)

In [ ]:
optimizers = update_optimizers(3, optimizers)
plot_gp(optimizers, xx, target, params, 5)

In [ ]:
optimizers = update_optimizers(3, optimizers)
plot_gp(optimizers, xx, target, params, 5)

In [ ]:
optimizers = update_optimizers(3, optimizers)
plot_gp(optimizers, xx, target, params, 5)

2D, without noise

In [ ]:
def grid_construction(pbounds, n_grid):

    dim = pbounds.shape[0]
    init = [] #[np.zeros(n_grid)] * dim

    for i in range(dim):
        init.append(np.linspace(pbounds[i,0], pbounds[i,1], n_grid)) #= np.linspace(pbounds[i,0], pbounds[i,1], n_grid) # works \forall p



    grid = np.meshgrid(*init) # list with p matrices n_grid x n_grid
    for g in range(len(grid)):
        grid[g] = grid[g].reshape(-1,1)
    grid = np.stack(grid, axis = -1)
    grid=grid[:,0,:] # array of shape (n_grid*n_grid, p): each row is composed of one element from init[0] and one element from init[1]
    return grid



def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1


optimizer1 = BayesianOptimization(
    f_temp=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
    )
optimizer2  = BayesianOptimization(
    f_temp=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
    )
optimizer3 = BayesianOptimization(
    f_temp=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
    )

grid = grid_construction(optimizer1._space.bounds, 100) # PAY ATTENTION: the density of the grid influences the result of the regret
                                                        # Remember to take enough points

optimizer1.maximize(init_points=2, n_iter=0, kappa=2.5)
optimizer2.maximize(init_points=2, n_iter=0,acq='ei')
optimizer3.maximize(init_points=2, n_iter=0,acq='poi')
optimizers={'ucb': optimizer1, 'ei': optimizer2, 'poi': optimizer3}

params = {'ucb': {'kappa': 2.5, 'xi': 0}, 'ei': {'kappa': 5, 'xi': 0}, 'poi': {'kappa': 5,'xi': 0.01}}
plot_simple_regret(optimizers, grid, black_box_function, params, dim=2)